In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet, MobileNetV2
from keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
import itertools
import os
import shutil
import random
from sklearn.metrics import confusion_matrix
import datetime
from numpy.random import seed
seed(1)

In [41]:
#Second way to load the data
#If we want the data to be broken up into Train, validation and Testing
# Change Benign and Malignant to 0 and 1 resp
os.chdir('C:\Sujata\CompScience\Winter 2021\Deep Learning\Project Final\images')
if os.path.isdir('train/0/') is False:
    os.mkdir('train')
    os.mkdir('valid')
    os.mkdir('test')
    random.seed(2021)
    
    for i in range(0, 2):
        shutil.move(f'{i}', 'train')
        os.mkdir(f'valid/{i}')
        os.mkdir(f'test/{i}')
        
        valid_samples = random.sample(os.listdir(f'train/{i}'), 788)
        for j in valid_samples:
            shutil.move(f'train/{i}/{j}', f'valid/{i}')
            
        test_samples = random.sample(os.listdir(f'train/{i}'), 394)
        for k in test_samples:
            shutil.move(f'train/{i}/{k}', f'test/{i}')
            
os.chdir('../..')
    

In [42]:
# train/valid/test - 70/20/10
train_path = 'C:/Sujata/CompScience/Winter 2021/Deep Learning/Project Final/images/train'
valid_path = 'C:/Sujata/CompScience/Winter 2021/Deep Learning/Project Final/images/valid'
test_path = 'C:/Sujata/CompScience/Winter 2021/Deep Learning/Project Final/images/test'

train_batch = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory = train_path, target_size = (224,224), batch_size = 10)
valid_batch = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory = valid_path, target_size = (224,224), batch_size = 10)
test_batch = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory = test_path, target_size = (224,224), batch_size = 10, shuffle = False)

Found 5545 images belonging to 2 classes.
Found 1576 images belonging to 2 classes.
Found 788 images belonging to 2 classes.


In [7]:
# input the pretrained model from keras
mobile = keras.applications.MobileNet()

In [8]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

In [9]:
# input the pretrained model from keras without the FC layers 
x = mobile.layers[-6].output # discarding the last 6 layers
output = Dense(units = 2, activation = 'softmax')(x) # Adding the dense layer and softmax activation
model = Model(inputs = mobile.input, outputs=output) # Making a new model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [10]:
#Freezing all layers except the last 2 layers
for layer in model.layers[:-2]:
    layer.trainable = False
    
#model.summary()

In [30]:
model.compile(optimizer = Adam(lr=0.0001), loss ='categorical_crossentropy', metrics = ['accuracy'])

In [31]:
# Clear any logs from previous runs
rm -rf ./logs/

path = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorBoard = tf.keras.callbacks.TensorBoard(log_dir=path, histogram_freq=0, write_graph=True,
                                                          write_grads=False, write_images=False, embeddings_freq=0,
                                                          embeddings_layer_names=None, embeddings_metadata=None,
                                                          embeddings_data=None, update_freq=50)

CheckPoint = tf.keras.callbacks.ModelCheckpoint(filepath='/tmp/weights.hdf5', monitor='val_loss', verbose=0,
                                                               save_best_only=False, save_weights_only=False, mode='auto',
                                                               save_freq=2000)
callbacks = [CheckPoint, tensorBoard]

In [32]:
model.fit(x=train_batch, validation_data = valid_batch, epochs = 10 , verbose = 2, callbacks = callbacks)

Epoch 1/10
555/555 - 144s - loss: 0.3267 - accuracy: 0.8651 - val_loss: 0.5162 - val_accuracy: 0.7633
Epoch 2/10
555/555 - 154s - loss: 0.3093 - accuracy: 0.8759 - val_loss: 0.4943 - val_accuracy: 0.7766
Epoch 3/10
555/555 - 146s - loss: 0.2961 - accuracy: 0.8812 - val_loss: 0.4874 - val_accuracy: 0.7779
Epoch 4/10
555/555 - 151s - loss: 0.2852 - accuracy: 0.8876 - val_loss: 0.4690 - val_accuracy: 0.7874
Epoch 5/10
555/555 - 155s - loss: 0.2756 - accuracy: 0.8929 - val_loss: 0.4707 - val_accuracy: 0.7855
Epoch 6/10
555/555 - 174s - loss: 0.2684 - accuracy: 0.8947 - val_loss: 0.4811 - val_accuracy: 0.7830
Epoch 7/10
555/555 - 176s - loss: 0.2616 - accuracy: 0.8977 - val_loss: 0.4317 - val_accuracy: 0.8109
Epoch 8/10
555/555 - 176s - loss: 0.2552 - accuracy: 0.9001 - val_loss: 0.4403 - val_accuracy: 0.8077
Epoch 9/10
555/555 - 176s - loss: 0.2498 - accuracy: 0.9026 - val_loss: 0.4575 - val_accuracy: 0.7970
Epoch 10/10
555/555 - 179s - loss: 0.2445 - accuracy: 0.9041 - val_loss: 0.4082 - 

In [33]:
%load_ext tensorboard
%tensorboard --logdir logs/fit/

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 13040.